In [160]:
import sys
import fitz
import requests
import json
import numpy as np

# Newer tests with rect-based tagging

Should in theory be more robust and less error prone.

In [161]:
red    = (1, 0, 0)
blue   = (0, 0, 1)
gold   = (1, 1, 0)
colors = {"stroke": blue, "fill": gold}
colors2 = {"fill": blue, "stroke": gold}
border = {"width": 0.3, "dashes": [2]}

In [166]:
color_dict = {
 'art': red,
 'eve': red,
 'geo': gold,
 'gpe': gold,
 'nat': gold,
 'org': (171/255, 255/255, 138/255),
 'per': (161/255, 255/255, 244/255),
 'tim': (247/255, 202/255, 111/255)
}

In [167]:
def mark_words_with_rect(page,tags) : 

    tokens = tags['tokens']
    rects  = tags['rects']

    for i in range(len(tokens)) :
        for j in range(len(tokens[i])) : 
            
            value = tokens[i][j]['token']
            label = tokens[i][j]['label']
            rect  = [float(ri) for ri in rects[i][j]]
            
            if label == 'O' :
                continue
            
            r = fitz.Rect(rect)       # make rect from word bbox
            annot = page.addHighlightAnnot(rect)
            annot.setInfo(info="dsfhdfsdhfsh",content="label: %s, value: %s"%(label[2:],value))
            annot.setBorder(border)
            annot.setColors({"stroke": color_dict[label[2:]], "fill": color_dict[label[2:]]})
            annot.update()

In [168]:
fname = "doc2.pdf"
doc   = fitz.open(fname)

for page in doc :
    
    # NOTE: getTextWords also returns rect - we can use this probably
    sentence = " ".join([word[4] for word in page.getTextWords()])
    print(sentence)
    
    npp    = np.array(page.getTextWords())
    words  = npp[:,4].tolist()
    rects  = npp[:,0:4].tolist()
    
    url   = 'http://127.0.0.1:5050/tag_sentence_rects'
    myobj = {'tokens': words, 'rects' : rects}
    print(myobj)
    x      = requests.post(url=url,json=myobj)
    tags   = json.loads(x.text)['result']
    
    print(tags)
    
    mark_words_with_rect(page, tags)
    
doc.save("marked-" + doc.name)

The origins of this company are in Sapporo, Hokkaido, during the Meiji period, where the Hokkaido Development Commission (Kaitakushi) established many businesses. Seibei Nakagawa, a German-trained brewer, became the ﬁrst brewmaster of the Kaitakushi Brewery in June 1876, and the ﬁrst Sapporo Lager was produced at that time. Privatized in 1886, the Sapporo brewery became the centerpiece for the Sapporo Beer Company. In 1887, another company, the Japan Beer Brewery Company was established in Mita, Meguro, Tokyo, and began producing Yebisu Beer. The competition between Sapporo and Japan Beer, as well as competition with the Osaka (now Asahi) and Kirin breweries led to a 1906 merger of Sapporo, Japan, and Osaka breweries into the Dai-Nippon Beer Company, Ltd., which formed a near monopoly on the Japanese market until after World War II. After 1949, Dai-Nippon was split into Nippon and Asahi breweries, with the Nippon Breweries resuming production of Sapporo beer in 1956 and renaming itself

{'rects': [[['72.0', '73.52978515625', '90.95467376708984', '84.52978515625'], ['94.01100158691406', '73.52978515625', '126.41700744628906', '84.52978515625'], ['129.47280883789062', '73.52978515625', '138.64715576171875', '84.52978515625'], ['141.70260620117188', '73.52978515625', '158.82080078125', '84.52978515625'], ['161.87661743164062', '73.52978515625', '206.51239013671875', '84.52978515625'], ['209.5681915283203', '73.52978515625', '225.46705627441406', '84.52978515625'], ['228.52337646484375', '73.52978515625', '237.0852508544922', '84.52978515625'], ['240.14157104492188', '73.52978515625', '284.7886962890625', '84.52978515625'], ['240.14157104492188', '73.52978515625', '284.7886962890625', '84.52978515625'], ['287.8441467285156', '73.52978515625', '336.761474609375', '84.52978515625'], ['287.8441467285156', '73.52978515625', '336.761474609375', '84.52978515625'], ['339.81695556640625', '73.52978515625', '370.3963928222656', '84.52978515625'], ['373.4527282714844', '73.52978515

In [132]:
tags['tokens'][1][4],tags['rects'][1][4]

({'label': 'B-tim', 'token': '1886'},
 ['460.9115905761719',
  '112.52978515625',
  '488.4405212402344',
  '123.52978515625'])

<hr>

# Older tests with string-based tagging

In [37]:
def mark_word(page, text) :
    """Underline each word that contains 'text'.
    """
    found = 0
    wlist = page.getTextWords()        # make the word list
    for w in wlist:                    # scan through all words on page
        print(w)
        if text in w[4]:               # w[4] is the word's string
            found += 1                 # count
            r = fitz.Rect(w[:4])       # make rect from word bbox
            annot = page.addHighlightAnnot(w[:4])
            annot.setInfo(info="dsfhdfsdhfsh",content="value: %s"%(text))
            annot.update()
            
            #page.addUnderlineAnnot(r)  # underline
    return found

In [97]:
def is_punct(s) :
    for p in './()[]$!,$%#-' :
        if s==p :
            return True
    return False

def matches(token,word) :
    if token.lower()==word.lower() :
        return True
    if token.lower()==word[0:-1].lower() and is_punct(word[-1]) :
        return True
    return False
    
def mark_words(page, sentences) :
    """
    Create annotations wherever 
    """
    found = 0
    
    # make the word list
    wlist = page.getTextWords()        
    
    # Create a dict for keeping track of which
    # tokens have already been seen
    seen_tokens = {}
    for tokens in sentences : 
        for token in tokens :
            seen_tokens[token['token']] = -1

    for tokens in sentences : 
        print("new sentence")
        for token in tokens :
            # Find the token in the text
            page.searchFor(token,hit_max=100000)
            
            # 
            
            
#             for i in range(len(wlist)) :
            # for w in wlist :
#                 w = wlist[i]
#                 if matches(token['token'],w[4]) and i>seen_tokens[token['token']] :
#                     print("Found match: %s, %s"%(token['token'],w[4]))
#                     if token['label'] != 'O' :
#                         r = fitz.Rect(w[:4])       # make rect from word bbox
#                         annot = page.addHighlightAnnot(w[:4])
#                         annot.setInfo(content="type: %s value: %s"%(token['label'], w[4]))
#                         annot.update()
#                     seen_tokens[token['token']] += 1
#                     break

In [98]:
page.searchFor("after 1949")

[Rect(186.32958984375, 216.52978515625, 237.0874481201172, 227.52978515625)]

In [96]:
fname = "doc2.pdf"
doc   = fitz.open(fname)

# new_doc = False                        # indicator if anything found at all
for page in doc:                       # scan through the pages
    
    # NOTE: getTextWords also returns rect - we can use this probably
    sentence = " ".join([word[4] for word in page.getTextWords()])

#     print(page.searchFor("asahi"))
#     continue
    
    url   = 'http://127.0.0.1:5050/tag_sentence'
    myobj = {'sentence': sentence}
    print(myobj)
    x      = requests.post(url=url,json=myobj)
    
    mark_words(page, json.loads(x.text)['result'])
    
doc.save("marked-" + doc.name)
    
#     found = mark_word(page, text)      # mark the page's words
#     if found:                          # if anything found ...
#         new_doc = True
#         print("found '%s' %i times on page %i" % (text, found, page.number + 1))

# if new_doc:
#     doc.save("marked-" + doc.name)

[Rect(72.0, 190.52978515625, 99.51725006103516, 201.52978515625), Rect(426.0403747558594, 216.52978515625, 453.5576171875, 227.52978515625)]


In [112]:
len(np.array(page.getTextWords())[:,0:4])

316

In [113]:
len(np.array(page.getTextWords())[:,4])

316

In [115]:
page.getTextWords()

[(72.0, 73.52978515625, 90.95467376708984, 84.52978515625, 'The', 0, 0, 0),
 (94.01100158691406,
  73.52978515625,
  126.41700744628906,
  84.52978515625,
  'origins',
  0,
  0,
  1),
 (129.47280883789062,
  73.52978515625,
  138.64715576171875,
  84.52978515625,
  'of',
  0,
  0,
  2),
 (141.70260620117188,
  73.52978515625,
  158.82080078125,
  84.52978515625,
  'this',
  0,
  0,
  3),
 (161.87661743164062,
  73.52978515625,
  206.51239013671875,
  84.52978515625,
  'company',
  0,
  0,
  4),
 (209.5681915283203,
  73.52978515625,
  225.46705627441406,
  84.52978515625,
  'are',
  0,
  0,
  5),
 (228.52337646484375,
  73.52978515625,
  237.0852508544922,
  84.52978515625,
  'in',
  0,
  0,
  6),
 (240.14157104492188,
  73.52978515625,
  284.7886962890625,
  84.52978515625,
  'Sapporo,',
  0,
  0,
  7),
 (287.8441467285156,
  73.52978515625,
  336.761474609375,
  84.52978515625,
  'Hokkaido,',
  0,
  0,
  8),
 (339.81695556640625,
  73.52978515625,
  370.3963928222656,
  84.5297851562

In [114]:
np.array(page.getTextWords())[:,4]

array(['The', 'origins', 'of', 'this', 'company', 'are', 'in', 'Sapporo,',
       'Hokkaido,', 'during', 'the', 'Meiji', 'period,', 'where', 'the',
       'Hokkaido', 'Development', 'Commission', '(Kaitakushi)',
       'established', 'many', 'businesses.', 'Seibei', 'Nakagawa,', 'a',
       'German-trained', 'brewer,', 'became', 'the', 'ﬁrst', 'brewmaster',
       'of', 'the', 'Kaitakushi', 'Brewery', 'in', 'June', '1876,', 'and',
       'the', 'ﬁrst', 'Sapporo', 'Lager', 'was', 'produced', 'at', 'that',
       'time.', 'Privatized', 'in', '1886,', 'the', 'Sapporo', 'brewery',
       'became', 'the', 'centerpiece', 'for', 'the', 'Sapporo', 'Beer',
       'Company.', 'In', '1887,', 'another', 'company,', 'the', 'Japan',
       'Beer', 'Brewery', 'Company', 'was', 'established', 'in', 'Mita,',
       'Meguro,', 'Tokyo,', 'and', 'began', 'producing', 'Yebisu',
       'Beer.', 'The', 'competition', 'between', 'Sapporo', 'and',
       'Japan', 'Beer,', 'as', 'well', 'as', 'competition', 'wi